In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime

import ccxt

from tp_utils.data_provider import read_prices, read_data, market_to_symbol, load_data_from_exchange
from Portfolio.portfolio_tools import *
from tp_utils.tp_utils import  create_market_buy_order,  create_market_sell_order, get_balance
from tp_config import *

pd.set_option("display.max_rows", 90)

In [50]:
exchange = ccxt.binance({'apiKey': main_api, 'secret': main_secret})

def get_balance_values(df, assets):
    if type(assets) == list :
        if len(assets) > 0:
            df = df[df['asset'].isin(assets)]
            df = df.set_index('asset').loc[assets].reset_index()
            res = df['free'].values.astype(float).tolist()
        else:
            res = []
    elif type(assets) == str :
        res = df[df['asset'] == assets]['free'].values.astype(float)[0]
    else:
        res = None
    return res
  

In [2]:
a = 10
a*(3,2)[a<0]

30

## Balance

In [44]:
df = get_balance(exchange)
df = df[(df['free'].astype(float) +df['locked'].astype(float)) > 0]
df = df[~df['asset'].isin(delisted_assets)]
res_balance = []
df = df.sort_values('asset')

ticker = 'BTC/USDT'
price_btc_usdt = exchange.fetch_ticker(ticker)['last']

balance_usdt = 0
balance_btc = 0
balance_alts = 0
sums = []
for row in df.iterrows():
    asset = row[1]['asset']
    amount = float(row[1]['free']) + float(row[1]['locked'])  
    if asset == 'BTC':
        sum_btc = amount
        balance_btc = balance_btc + amount 
        balance_usdt = balance_usdt + amount * price_btc_usdt
    elif asset == "USDT":
        balance_usdt = balance_usdt + amount
        sum_btc = 0
    else:
        ticker = asset + '/BTC'
        price_btc = exchange.fetch_ticker(ticker)['last']
        balance_btc = balance_btc + amount * price_btc
        if asset != 'BNB':
            balance_alts = balance_alts + amount * price_btc     
            
        balance_usdt = balance_usdt + amount * price_btc * price_btc_usdt
        sum_btc = round(amount * price_btc, 4)
    sums.append(sum_btc)
df['sum_btc'] = sums
print('BTC:', round (balance_btc,4), 'ALTs:', round(balance_alts,4)  ,'USDT:', round(balance_usdt))
df.sort_values("sum_btc", ascending = False)

AuthenticationError: bybit requires "apiKey" credential

In [36]:
#market = 'ADA-BTC'
#market = 'AVAX-BTC'
#market = 'ATA-BTC'
market = 'AXS-BTC'
#market = 'MATIC-BTC'
#market = 'FTT-BTC'

#market = 'XLM-BTC'
market = 'SOL-BTC'
#market = 'LINK-BTC'
#market = 'RUNE-BTC'
#market = 'BNB-BTC'
market = 'ETH-BTC'
#market = 'ETC-BTC'
#market = 'DOT-BTC'
#market = 'XMR-BTC'
#market = 'TRX-BTC'
#market = 'ZIL-BTC'
#market = 'VET-BTC'
#market = 'SUSHI-BTC'
#market = 'THETA-BTC'
#market = 'TFUEL-BTC'
#market = 'WAVES-BTC'
#market = 'SAND-BTC'
#market = 'ONE-BTC'
#market = 'UNI-BTC'
#market = 'ZRX-BTC'
#market = 'TLM-BTC'
#market = 'DOGE-BTC'
#market = 'ALICE-BTC'
#market = 'LUNA-BTC'
#market = 'RUNE-BTC'
#market = 'BQX-BTC'
#market = 'YFII-BTC'
amount = 0.2
symbol = market_to_symbol(market)

## Market buy

In [37]:
price_buy = create_market_buy_order(exchange, market, amount)
print('price', price_buy, ', sum', price_buy * amount)

price 0.071306 , sum 0.0142612


## Market Sell

In [15]:
price_sell = create_market_sell_order(exchange, market, amount)
print(price_sell, 'sum', price_sell * amount)

0.00018281 sum 0.018281


### Limit sell

In [21]:
order_type = 'stop_loss_limit'  # or 'market', other types aren't unified yet
side = 'sell'
stop_price = 0.0002150
sell_price = 0.0002140
# your price
# overrides
params = {
    'stopPrice': stop_price,  # your stop price
}
order = exchange.create_order(symbol, order_type, side, amount, sell_price, params)

In [51]:
exchange.cancel_all_orders(symbol)

[{'info': {'symbol': 'DOGEBTC',
   'origClientOrderId': 'web_654918598cbd4910a3cb74fe677b92bf',
   'orderId': '239881844',
   'orderListId': '-1',
   'clientOrderId': 'SqiOk3B2fx3pQKDBISxZmL',
   'price': '0.00000578',
   'origQty': '553.00000000',
   'executedQty': '0.00000000',
   'cummulativeQuoteQty': '0.00000000',
   'status': 'CANCELED',
   'timeInForce': 'GTC',
   'type': 'STOP_LOSS_LIMIT',
   'side': 'SELL',
   'stopPrice': '0.00000579'},
  'id': '239881844',
  'clientOrderId': 'SqiOk3B2fx3pQKDBISxZmL',
  'timestamp': None,
  'datetime': None,
  'lastTradeTimestamp': None,
  'symbol': 'DOGE/BTC',
  'type': 'stop_loss_limit',
  'timeInForce': 'GTC',
  'postOnly': False,
  'side': 'sell',
  'price': 5.78e-06,
  'stopPrice': 5.79e-06,
  'amount': 553.0,
  'cost': 0.0,
  'average': None,
  'filled': 0.0,
  'remaining': 553.0,
  'status': 'canceled',
  'fee': None,
  'trades': [],
  'fees': []}]

# Sell All

In [56]:
df = get_balance(exchange)
df = df[(df['free'].astype(float) +df['locked'].astype(float)) > 0]
df = df[~df['asset'].isin(delisted_assets)]
for row in df.iterrows():
    asset = row[1]['asset']
    amount = row[1]['free']
    locked = row[1]['locked']
    if asset in ['BTC', 'USDT', 'BNB']:
        continue

    market = asset + '-BTC' 
    ticker = asset + '/BTC'
    if float(locked) > 0:
        exchange.cancel_all_orders(ticker)
        
    try:
        price_sell = create_market_sell_order(exchange, market, amount)
        print(market, price_sell)
    except:
        pass

ETH-BTC 0.069013
QTUM-BTC 0.0002347
DATA-BTC 2.81e-06
THETA-BTC 0.00015202
VET-BTC 2.34e-06
DOGE-BTC 5.64e-06
FTT-BTC 0.0010006
DREP-BTC 1.464e-05
SOL-BTC 0.00085663
DOT-BTC 0.00045237
UNI-BTC 0.0006086
PSG-BTC 0.0009337
TVK-BTC 7.17e-06


### Trades

In [988]:
raw_trades =  exchange.fetch_my_trades('TFUEL/BTC')
trades = [[round(x['timestamp']/1000), x['side'], x['amount'], x['price'], x['cost']] for x in raw_trades]

In [ ]:
df  = pd.DataFrame(trades, columns = ['time', 'side', 'amount', 'price', 'cost'])
dfx = df.groupby(['time', 'side']).sum()
dfx['price'] = dfx['cost']/dfx['amount']
dfx.sort_values('time', ascending = False)[:20]  

In [ ]:
portfolios = ['max_sharpe', 'main', 'index_low', 'efficient_risk']
for portfolio in portfolios:
    portfolio_file = portfolio + '.json'

    with open(portfolio_file) as f:
        portfolio_data = json.load(f)
    portfolio = portfolio_data['markets']
    sell_portfolio(exchange, portfolio)

In [327]:
d = {"stop":not True}
with open("stop.json", 'w') as f:
    json.dump(d, f)

In [328]:
with open("stop.json") as f:
    x = json.load(f)
x

{'stop': False}

## Portfolio recalc

### 0. Data from exchange

In [41]:
pair = 'USDT'
index_assets = pd.read_csv('Portfolio/index_assets.csv')['asset'].tolist()
markets_index = [s + '-' + pair for s in index_assets]

index_low_assets = pd.read_csv('Portfolio/index_low_assets.csv')['asset'].tolist()
index_all = index_assets + index_low_assets
markets = [s + '-' + pair for s in index_all]

In [ ]:
exchange = ccxt.bybit()
pair = 'USDT'
assets = ['ADA', 'APT', 'AVAX', 'BNB', 'SOL', 'TRX', 'ZIL', 'NEON',  'WLD', 'PEPE', 'NEAR', 'ETH', 'BONK', 'WAVES',
          'FTT',  'FTM', 'MATIC', 'INJ', 'JUP', 'SEI', 'SUI', 'THETA', 'RUNE', 'GALA', 'PYTH', 'PPT', 'FIRE', 'NGL',
         'CYBER']

assets.sort()

#markets = [m + '-' + pair for m in assets]

start_timestamp = exchange.parse8601('2023-04-01 00:00:00')
timeframes = {'1h':60, '4h':60}
data_path = DATA_PATH_CRYPTO + '/' + pair
print(data_path)
load_data_from_exchange(exchange, markets, timeframes, start_timestamp, data_path, 10)

### 1. Set index assets

In [56]:
import os

portfolio = 'max_sharpe'
#portfolio = 'main'
portfolio = 'index_low'
#portfolio = 'efficient_risk'

portfolio_file = portfolio + '.json'

if os.path.exists(portfolio_file):
    with open(portfolio_file) as f:
        portfolio_data = json.load(f)
        opt_method = portfolio_data['opt_method']
        
        if portfolio_data['base'][0] == 'index':
            assets = index_assets
        elif portfolio_data['base'][0] == 'index_low':
            assets = index_low_assets    
        elif portfolio_data['base'][0] == 'index_all':
            assets = index_all          

opt_method = "max_sharpe"
pair = 'USDT'
#pair = 'BTC'


In [38]:
def print_data(df):
    print(datetime.fromtimestamp(df.index[0]/1000))
    print(datetime.fromtimestamp(df.index[-1]/1000))

### 2. Load data

In [42]:
df_prices =  load_data_for_portfolio(markets_index, '4h')   
print(datetime.fromtimestamp(df_prices.index[-1]/1000))

AVAX-USDT
BTC-USDT
BNB-USDT
DOT-USDT
ETH-USDT
LINK-USDT
LTC-USDT
LUNA-USDT
MATIC-USDT
SOL-USDT
TON-USDT
TRX-USDT
2024-03-24 08:00:00


### 3. Calc weights

#### 3.1 Calc weights 

In [43]:
lookback = 120
df_period = df_prices.iloc[-lookback:]
print_data(df_period)

ef = calc_frontier(df_period, "ledoit_wolf",  "ema_historical_return", span = 150)
#ef = calc_frontier(df_period, 'semicovariance', "ema_historical_return",)


#opt_method = "efficient_risk"
opt_method = 'max_sharpe'

if opt_method == 'max_sharpe': 
    dfw = calc_weights(ef, opt_method, 0)
else:
    dfw = calc_weights(ef, opt_method, 0.25)
print(opt_method)    
dfw 

2024-03-04 12:00:00
2024-03-24 08:00:00
max_sharpe


,W,weights
LUNA-USDT,0.04239,0.04239
TON-USDT,0.95761,0.95761


### 3.2 Calc weghts min. cardinality

In [40]:
import riskfolio as rp

rms= ['MV', 'MSV',  'CVaR']

def riskfolio_weights(df_period, rm , obj):
    """
        obj - Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    """
    Y = df_period.pct_change().dropna()

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)

    # Calculating optimum portfolio

    # Select method and estimate input parameters:

    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Estimate optimal portfolio:

    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    # First we need to delete the cardinality constraint
 #   port.card = None 

    # Then we need to set the constraint on the minimum number of effective assets
  #  port.nea = 2
    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    w = w[w.weights > 0.01]
    return w

print_data(df_period)
ws = pd.DataFrame([])
for rm in rms:
    w = riskfolio_weights(df_period, rm, 'MaxRet')
    print(w)

2024-03-04 12:00:00
2024-03-24 08:00:00
You must convert self.cov to a positive definite matrix
            weights
TOKEN-USDT      1.0
You must convert self.cov to a positive definite matrix
            weights
TOKEN-USDT      1.0
You must convert self.cov to a positive definite matrix
            weights
TOKEN-USDT      1.0


In [25]:
# First we need to delete the cardinality constraint
port.card = None 

# Then we need to set the constraint on the minimum number of effective assets
port.nea = 2

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w[w.weights > 0.01])

,weights
ETHFI-USDT,0.016718167
ONDO-USDT,0.462106652
TOKEN-USDT,0.521174797


In [28]:
# Number of assets in portfolio
n_assets = np.sum(np.where(np.round(w,4) > 0, 1, 0)).item()

# Number of effective assets in portfolio
nea = 1/np.sum(w ** 2).item()

print('Number of Assets:', n_assets)
print('Number of Effective Assets:', nea)

Number of Assets: 14
Number of Effective Assets: 9.99999858246235


D:\Soft\Anaconda\envs\t\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


### 4. Sell portfolio

In [18]:
portfolio = portfolio_data['markets']
balance = sell_portfolio(portfolio)
print('Current balance:', balance)  

{'opt_method': 'max_sharpe', 'base': ['index'], 'markets': {'ADA-BTC': 647.583133356188, 'AVAX-BTC': 19.2435189120272, 'DOGE-BTC': 1595.947219604147, 'ETC-BTC': 17.649557037172347, 'ONT-BTC': 105.2389705882353, 'RUNE-BTC': 8.051865907653385, 'XMR-BTC': 0.5688474256022674}}
Current balance: 0.09769360876527733


In [39]:
balance = 0.2

### 5. Set new portfolio

In [40]:
portfolio = set_new_portfolio(exchange,dfw, balance)
portfolio

{'ADA-BTC': 1220.7858048162232,
 'CTSI-BTC': 1544.097807757167,
 'ETC-BTC': 33.56746946583607,
 'ETH-BTC': 0.34019285042333025,
 'ONT-BTC': 491.70230966638155,
 'RUNE-BTC': 15.038899247733559,
 'LTC-BTC': 0.41952,
 'XVG-BTC': 2419.512195121951}

### 6. Buy portfolio

In [41]:
for market in portfolio.keys():
    amount = portfolio[market]
    print(market, amount)
    price_buy =  create_market_buy_order(exchange, market, amount)

portfolio_data['markets'] = portfolio    
with open(portfolio_file, 'w') as f:
    json.dump(portfolio_data, f)    

ADA-BTC 1220.7858048162232
CTSI-BTC 1544.097807757167
ETC-BTC 33.56746946583607
ETH-BTC 0.34019285042333025
ONT-BTC 491.70230966638155
RUNE-BTC 15.038899247733559
LTC-BTC 0.41952
XVG-BTC 2419.512195121951


### End

## Stop limit orders

In [27]:
side = 'sell'
stop_price = 6
limit_price = 6920
params = {'stopPrice': stop_price }
order =  exchange.createOrder(symbol, 'STOP_LOSS_LIMIT', side, amount, limit_price, params)
print(order['id'])

1714800797


In [ ]:
inf = exchange.cancel_order(order['id'], symbol)

In [ ]:
#Типы поддерживаемых ордеров

from pprint import pprint
import ccxt
exchange = ccxt.binance()
exchange.load_markets()
pprint(exchange.markets['BTC/USDT'])

## Skew

In [3]:
from scipy.stats import skew, skewtest

In [1]:
timeframes = ['1m', '3m', '5m',  '15m', '30m', '1h', '2h']

In [20]:
skew_list = []
for tf in timeframes:
    prices = read_data('BTC-USDT', tf)
    dret = daily_returns(prices['C'])
    skew_list.append([tf, skew(dret)])
    
df_skew = pd.DataFrame(skew_list)    
df_skew.columns = ['tf', 'skew']    

In [21]:
skew_treshold = 0.15
df_skew[df_skew['skew'] > skew_treshold]

,tf,skew
0,1m,0.294967222
1,3m,0.675015277
2,5m,0.355812883
4,30m,0.404607105
5,1h,0.207761672
7,4h,0.834205604
8,6h,0.556794138


In [126]:
round(10.23234234, 2)

10.23